In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

# 기상청 날씨 데이터 전처리
data2016 = pd.read_csv('./data/rawdata/기상청/2016_01_to_12.csv', encoding='euc-kr', index=False)
data2017 = pd.read_csv('./data/rawdata/기상청/2017_01_to_12.csv', encoding='euc-kr', index=False)
data2018 = pd.read_csv('./data/rawdata/기상청/2018_01_to_12.csv', encoding='euc-kr', index=False)
data2019 = pd.read_csv('./data/rawdata/기상청/2019_01_to_12.csv', encoding='euc-kr', index=False)
data2020 = pd.read_csv('./data/rawdata/기상청/2020_01_to_12.csv', encoding='euc-kr', index=False)
data2021 = pd.read_csv('./data/rawdata/기상청/2021_01_to_07.csv', encoding='euc-kr', index=False)

train = pd.read_csv('./data/rawdata/train.csv')

merge_df = pd.concat([data2016, data2017, data2018, data2019, data2020, data2021])

# 필요 없는 컬럼 제거
merge_df.drop(['지점', '지점명', '기온 QC플래그', '강수량 QC플래그',  '풍속 QC플래그', '풍향(16방위)',
               '풍향 QC플래그', '습도 QC플래그', '증기압(hPa)', '이슬점온도(°C)',  '3시간신적설(cm)'], axis=1, inplace=True)

# 중식, 석식 시간대 데이터 필터링
mask_d = merge_df['일시'].str.slice(start=10).str.strip() =='18:00'
mask_l = merge_df['일시'].str.slice(start=10).str.strip() == '12:00'

merge_df_d = merge_df.loc[mask_d,:]
merge_df_l = merge_df.loc[mask_l,:]

# 날짜 뽑아내기
merge_df_l['날짜'] = merge_df_l.iloc[:, 0].str[0:10]
merge_df_d['날짜'] = merge_df_d.iloc[:, 0].str[0:10]

# 날짜형으로 형변환
merge_df_l['날짜'] = pd.to_datetime(merge_df_l.iloc[:,6])
merge_df_d['날짜'] = pd.to_datetime(merge_df_d.iloc[:,6])

# 요일 추가
merge_df_l['요일'] = merge_df_l['날짜'].dt.dayofweek
merge_df_d['요일'] = merge_df_d['날짜'].dt.dayofweek

# 결측치 제거
merge_df_l = merge_df_l.fillna(0)
merge_df_d = merge_df_d.fillna(0)

# 평일만 필터링
mask_l = (merge_df_l['요일'] != 5) & (merge_df_l['요일'] != 6)
mask_d = (merge_df_d['요일'] != 5) & (merge_df_d['요일'] != 6)

merge_df_l = merge_df_l.loc[mask_l, :]
merge_df_d = merge_df_d.loc[mask_d, :]

# 요일 매핑
merge_df_l['요일'] = merge_df_l['요일'].map({0:'월', 1:'화',2:'수',3:'목',4:'금'})
merge_df_d['요일'] = merge_df_d['요일'].map({0:'월', 1:'화',2:'수',3:'목',4:'금'})



TypeError: read_csv() got an unexpected keyword argument 'index'

In [14]:
merge_df_l.reset_index(drop=False, inplace=True)

In [15]:
merge_df_d.reset_index(drop=False, inplace=True)

In [28]:
merge_df_l.drop('index', axis=1, inplace=True)

In [29]:
merge_df_d.drop('index', axis=1, inplace=True)

In [30]:
merge_df_d

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),날짜,요일
0,2016-01-01 18:00,4.4,0.0,1.0,57.0,0.0,2016-01-01,금
1,2016-01-04 18:00,8.1,0.0,0.5,32.0,0.0,2016-01-04,월
2,2016-01-05 18:00,1.6,0.0,0.6,72.0,0.0,2016-01-05,화
3,2016-01-06 18:00,5.2,0.0,0.3,41.0,0.0,2016-01-06,수
4,2016-01-07 18:00,3.4,0.0,0.5,35.0,0.0,2016-01-07,목
...,...,...,...,...,...,...,...,...
1450,2021-07-23 18:00,30.3,0.0,1.8,72.0,0.0,2021-07-23,금
1451,2021-07-26 18:00,29.5,0.0,2.4,63.0,0.0,2021-07-26,월
1452,2021-07-27 18:00,30.1,0.0,1.2,71.0,0.0,2021-07-27,화
1453,2021-07-28 18:00,30.5,0.0,1.4,69.0,0.0,2021-07-28,수


In [37]:
# 날짜 필터링

train_date = train['일자']
train_date

0       2016-02-01
1       2016-02-02
2       2016-02-03
3       2016-02-04
4       2016-02-05
           ...    
1200    2021-01-20
1201    2021-01-21
1202    2021-01-22
1203    2021-01-25
1204    2021-01-26
Name: 일자, Length: 1205, dtype: object

In [40]:
# mask_date1 = (merge_df_l['날짜'].map(tuple).isin[train_date])
# train['일자']
test_mask1 = merge_df_l['날짜'].isin(train_date)
test_ma = merge_df_l.loc[test_mask1, :]

test_mask2 = merge_df_d['날짜'].isin(train_date)
test_ma = merge_df_l.loc[test_mask2, :]

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),날짜,요일
21,2016-02-01 12:00,3.0,0.0,1.9,24.0,0.0,2016-02-01,월
22,2016-02-02 12:00,1.7,0.0,1.6,24.0,0.0,2016-02-02,화
23,2016-02-03 12:00,3.2,0.0,1.5,31.0,0.0,2016-02-03,수
24,2016-02-04 12:00,3.7,0.0,0.7,30.0,0.0,2016-02-04,목
25,2016-02-05 12:00,6.3,0.0,1.0,17.0,0.0,2016-02-05,금
...,...,...,...,...,...,...,...,...
1318,2021-01-20 12:00,4.8,0.0,1.2,31.0,0.0,2021-01-20,수
1319,2021-01-21 12:00,9.1,0.0,0.4,52.0,0.0,2021-01-21,목
1320,2021-01-22 12:00,8.4,0.0,0.5,94.0,0.0,2021-01-22,금
1321,2021-01-25 12:00,11.3,0.0,2.6,54.0,0.0,2021-01-25,월


In [9]:
merge_df_l['날짜'] 

11     2016-01-01
83     2016-01-04
107    2016-01-05
131    2016-01-06
155    2016-01-07
          ...    
4883   2021-07-23
4955   2021-07-26
4979   2021-07-27
5003   2021-07-28
5027   2021-07-29
Name: 날짜, Length: 1455, dtype: datetime64[ns]

In [ ]:
# 날짜 필터링

train_date = train['일자']

mask_date1 = (merge_df_l['날짜'] >= '2016-02-01')&(merge_df_l['날짜'] <= '2021-01-26')  
mask_date2 = (merge_df_d['날짜'] >= '2016-02-01')&(merge_df_d['날짜'] <= '2021-01-26')  


In [ ]:

merge_df_l = merge_df_l.loc[mask_date1, :]
merge_df_d = merge_df_d.loc[mask_date2, :]

date_train_d_mask = merge_df_d['날짜'] <= '2020-10-31'
date_test_d_mask = merge_df_d['날짜'] > '2020-10-31'
date_train_l_mask = merge_df_l['날짜'] <= '2020-10-31'
date_test_l_mask = merge_df_l['날짜'] > '2020-10-31'

date_train_d = merge_df_d.loc[date_train_d_mask, :]
date_test_d = merge_df_d.loc[date_test_d_mask, :]
date_train_l = merge_df_l.loc[date_train_l_mask, :]
date_test_l = merge_df_l.loc[date_test_l_mask, :]

date_test_d.drop('일시', axis=1, inplace=True)
date_train_d.drop('일시', axis=1, inplace=True)
date_test_l.drop('일시', axis=1, inplace=True)
date_train_l.drop('일시', axis=1, inplace=True)

date_test_d = date_test_d[["날짜",'요일','기온(°C)','강수량(mm)','풍속(m/s)',"습도(%)",'적설(cm)']]
date_train_d = date_train_d[["날짜",'요일','기온(°C)','강수량(mm)','풍속(m/s)',"습도(%)",'적설(cm)']]
date_test_l = date_test_l[["날짜",'요일','기온(°C)','강수량(mm)','풍속(m/s)',"습도(%)",'적설(cm)']]
date_train_l = date_train_l[["날짜",'요일','기온(°C)','강수량(mm)','풍속(m/s)',"습도(%)",'적설(cm)']]

date_train_d.to_csv('save/data/preprocess/date_train_dinner.csv', index=False)
date_test_d.to_csv('save/data/preprocess/date_test_dinner.csv', index=False)
date_train_l.to_csv('save/data/preprocess/date_train_lunch.csv', index=False)
date_test_l.to_csv('save/data/preprocess/date_test_lunch.csv', index=False)